In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import os 
for root, dirs, files in os.walk(os.getcwd()):
    for file in files: 
        if 'train.csv' in file: 
            path = os.path.join(root, file)


In [3]:
df=pd.read_csv(path)
df.sample(5)

,id,qid1,qid2,question1,question2,is_duplicate
357234,357234,486645,486646,What can I do rather than taking medicine when...,"As a COC player, what would you say if playing...",0
69697,69697,120230,120231,I am working at a tele networking company as a...,Can I get a job in Network companies if I choo...,0
272973,272973,391343,391344,How much investment reqiured for poultry in ka...,What were the fatal foreign policy mistakes th...,0
57845,57845,101596,101597,How do I know if a person who can provoke angr...,Which emerging technologies have the potential...,0
175186,175186,269789,134204,What steps should be taken for better startup ...,What steps are to be taken to begin a startup?,1


In [4]:
new_df=df.sample(30000)

new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [5]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
210776,210776,315428,315429,Where and how is the Rapture mentioned in the ...,Where is the Messiah mentioned in the Bible?,0
236884,236884,347913,347914,Which is the best NID coaching classes in Pune?,How did you feel when you realised you were a ...,0
277450,277450,396557,396558,How can you tell someone is in a gang?,How can I tell if someone is in a gang?,1
230906,230906,340575,164666,What is a good age to settle down and get marr...,When is a good time to get married for a man?,1
73349,73349,125868,125869,Why is the script in unity not allowing me to ...,Is my waist unusually wide for a man?,0


In [6]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
210776,Where and how is the Rapture mentioned in the ...,Where is the Messiah mentioned in the Bible?
236884,Which is the best NID coaching classes in Pune?,How did you feel when you realised you were a ...
277450,How can you tell someone is in a gang?,How can I tell if someone is in a gang?
230906,What is a good age to settle down and get marr...,When is a good time to get married for a man?
73349,Why is the script in unity not allowing me to ...,Is my waist unusually wide for a man?


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=3000)

In [8]:
questions=list(ques_df['question1']) + list(ques_df['question2'])
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [9]:
# x = np.arange(16.0).reshape(4, 4)
# print(x)
# print()
# print(np.vsplit(x, 2))

In [15]:
temp_df1 = pd.DataFrame(q1_arr, index=ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index=ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2],axis=1)

In [16]:
temp_df.shape

(30000, 6000)

In [17]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [18]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
210776,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
236884,0,0,0,0,0,0,0,0,0,0,...,3,0,0,0,0,0,0,0,0,0
277450,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
230906,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
73349,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(temp_df.iloc[:,0:-1].values,
                                                    temp_df.iloc[:,-1].values, test_size=0.2,
                                                    random_state=1)

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test, y_pred)

0.7398333333333333

In [22]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

0.7315